In [ ]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

import os
import xarray as xr
import numpy as np
import pickle
import esiodata as ed
import itertools
import seaborn as sns


In [ ]:
dir_out = '/home/disk/sipn/nicway/python/ESIO/'

In [ ]:
# Create an esioData
E = ed.esiodata(data_dir = '/home/disk/sipn/nicway/data/')

In [ ]:
# Add observations
obs_list = ['G10006',  'NSIDC_0051',  'NSIDC_0079',  'NSIDC_0081']
for co in obs_list:
    E.add_obs(co, grid_file='stereo_gridinfo.nc')
E.add_obs('NSIDC_extent', grid_file='N/A')    

In [ ]:
# Add Models
E.add_model(mod_name='gfdlsipn', model_label='gfdl-flor', grid_file='GFDLFLOR_gridinfo.nc')
E.add_model(mod_name='piomas', model_label='piomas', grid_file='PIOMAS_gridinfo.nc')
E.add_model(mod_name='yopp', model_label='ecmwf-yopp', grid_file='yopp_land_sea_mask.nc')
# S2S download models
E.add_model(mod_name='bom', model_label='bom', grid_file='MISSING')
E.add_model(mod_name='cma', model_label='cma', grid_file='MISSING')
E.add_model(mod_name='ecmwf', model_label='ecmwf', grid_file='MISSING')
E.add_model(mod_name='hcmr', model_label='hcmr', grid_file='MISSING')
E.add_model(mod_name='isaccnr', model_label='isac-cnr', grid_file='MISSING')
E.add_model(mod_name='jma', model_label='jma', grid_file='MISSING')
E.add_model(mod_name='metreofr', model_label='météo-fr', grid_file='MISSING')
E.add_model(mod_name='ukmo', model_label='ukmo', grid_file='MISSING')
E.add_model(mod_name='eccc', model_label='eccc', grid_file='MISSING')
E.add_model(mod_name='kma', model_label='kma', grid_file='MISSING')
E.add_model(mod_name='ncep', model_label='ncep', grid_file='MISSING')
# c3S Models
E.add_model(mod_name='ukmetofficesipn', model_label='ukmo-c3s', grid_file='ukmetofficesipn_land_sea_mask.nc')
E.add_model(mod_name='ecmwfsipn', model_label='ecmwf-c3s', grid_file='ecmwfsipn_land_sea_mask.nc')
# NAVY NEMO
E.add_model(mod_name='usnavysipn', model_label='nrl', grid_file='MISSING')
E.add_model(mod_name='usnavyncep', model_label='nrl_subx', grid_file='MISSING')
# RASM ESRL
E.add_model(mod_name='rasmesrl', model_label='rasm-esrl', grid_file='MISSING')

In [ ]:
# Add other path info
E.fig_dir = r'/home/disk/sipn/nicway/public_html/sipn/figures/'

In [18]:
# Flag if modeled ice is predicted (True) or prescribed (False)
icePredicted = {'gfdlsipn':True, 'piomas':True, 'yopp':True, 'bom':False, 'cma':False, 'ecmwf':True, 
              'hcmr':False, 'isaccnr':False, 'jma':False, 'metreofr':True, 'ukmo':True, 'eccc':False, 
              'kma':True, 'ncep':True, 'ukmetofficesipn':True, 'ecmwfsipn':True, 'usnavysipn':True,
               'usnavyncep':True, 'rasmesrl':True}
E.icePredicted = icePredicted

In [19]:
# Check all models have a predicted flag
if not (icePredicted.keys() == E.model.keys()):
    raise ValueError("Missing some predicted flags for models...")

In [20]:
# Define colors/lines etce for each model to use in plots
cmap_c = itertools.cycle(sns.color_palette("Paired", len(E.model.keys()) ))
linecycler = itertools.cycle(["-","--","-.",":","--"])
markercycler = itertools.cycle(["*","o","s","v","x"])
mod_cc = {}
mod_ls = {}
mod_mk = {}
for (i, cmod) in enumerate(E.model.keys()):
    mod_cc[cmod] = next(cmap_c)
    mod_ls[cmod] = next(linecycler)
    mod_mk[cmod] = next(markercycler)
E.model_color = mod_cc
E.model_linestyle = mod_ls
E.model_marker= mod_mk

In [ ]:
# Save to ESIO dir
print("Saving to ", dir_out)
E.save()

In [ ]:
# # Load object to test
# E = None
# E = ed.esiodata.load()